In [18]:
import control as ctrl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
# ARX model of the boiler
A= np.array([ 1, -1.05169693, 0.07153242])
B = np.array([0.03318858, 0.09114499])

G = ctrl.TransferFunction(B, A, dt = 1.0)
G

TransferFunction(array([0.03318858, 0.09114499]), array([ 1.        , -1.05169693,  0.07153242]), 1.0)

In [ ]:
# Apply the ARMAX model to Heat schedule for a week
folder_path = r"D:\ms\January 2024\Thesis\Boiler-Bidding-model\Datasets\boiler-12025022508_00_17_LHP_2_PHLIT_export.csv"
testfile = pd.read_csv(folder_path)
testfile["Heat load forecast [MW]"] = testfile["Heat load forecast [MW]"].shift(-1)
testfile = testfile.iloc[:-1]

x_values = testfile['Heat load forecast [MW]']
x_values = x_values.values
time = np.arange(0, len(x_values), 1)

In [11]:
time, y_values =  ctrl.forced_response(G, time, x_values)

In [19]:
testfile['predicted MWh'] = y_values


fig = make_subplots(rows=1, cols=1,shared_xaxes=True,vertical_spacing=0.1,subplot_titles=('Time Series Plot'),specs=[[{'secondary_y': True}]]  )

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['Heat load forecast [MW]'], mode='lines', name='Heat load forecast [MW]'),secondary_y=False)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['L11 EK plan [MW]'], mode='lines', name='L11 EK plan [MW]'),secondary_y=False)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['L11 Heat storage energy [MWh]'], mode='lines', name='L11 Heat storage energy [MWh]'),secondary_y=True)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['predicted MWh'], mode='lines', name='predicted MWh'),secondary_y=True)

fig.update_layout(
    title_text="Time Series of Heat Load Forecast, EK Plan, and Heat Storage Energy",
    xaxis_title="UTC",
    yaxis_title="MW (Heat load forecast & EK plan)",
    yaxis2_title="MWh (Heat storage energy)",
    template="plotly",
)

fig.show()
